In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [5]:
#results_dir = "all_experiments"
results_dir = "../"
#sub_files = ["momentumTest", "chatAdamTests/checkpoints", "ScheduleTest_constant", "scheduleTest_all4", "1Test", "05Test", "01Test", "simpleCnnTests", "rdmExp_tests", "adrien", "adri_weights_no_reset", "TestTinyCNN", "TestTinyCnnAdam", 'TestTinyCnnAdam_all_input', "TestTinyCnnSgd_Input_all", "TestTinyCNN_every_layer_adam", "TestTinyCNN_every_layer_sgd"]
#sub_files = ["TMP_baby_stuck_noise", '1Test', '01Test']
#sub_files = ['Tests1/SGD', 'Tests1/M_SGD', 'Tests1/ADAM']
sub_files = ['Tests3/SGD', 'Tests3/M_SGD', 'Tests3/ADAM']
print(len(sub_files))
json_dir = [os.path.join(results_dir, sub_file, "metrics") for sub_file in sub_files]
json_dir += [os.path.join(results_dir, sub_file, "checkpoints/metrics") for sub_file in sub_files]

3


In [6]:
json_files = []
for dir_name in json_dir:
    if not os.path.exists(dir_name):
        print(f"Directory {dir_name} does not exist.")
    else:
        for root, dirs, files in os.walk(dir_name):
            for file in files:
                if file == "all_metrics.json":
                    json_files.append(os.path.join(root, file))
json_files

Directory ../Tests3/SGD/metrics does not exist.
Directory ../Tests3/M_SGD/metrics does not exist.
Directory ../Tests3/ADAM/metrics does not exist.


['../Tests3/SGD/checkpoints/metrics/resnet18_none_20250612_211807/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/resnet18_none_20250612_180636/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/simple_cnn_none_20250612_184818/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/resnet18_none_20250612_203242/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/baby_cnn_none_20250612_173623/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/tiny_cnn_none_20250612_201735/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/tiny_cnn_none_20250612_175136/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/tiny_cnn_none_20250612_200233/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/simple_cnn_none_20250612_224839/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/baby_cnn_none_20250612_193214/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/simple_cnn_none_20250612_220320/all_metrics.json',
 '../Tests3/SGD/checkpoints/metrics/baby_cnn_none_20250612_

In [7]:
def flatten_config(config):
    flat = {}
    for k, v in config.items():
        if isinstance(v, list):
            flat[k] = str(v)  # convertir liste en string
        else:
            flat[k] = v
    return flat

In [12]:
all_data = []
for json_file in json_files:
    print(f"Trying to open {json_file}")
    with open(json_file, 'r') as f:
        data = json.load(f)

        flat_conf = flatten_config(data["config"])

        flat_results = flatten_config(data['final_metrics'])
        merged = flat_conf | flat_results

        all_data.append(merged)

Trying to open ../Tests3/SGD/checkpoints/metrics/resnet18_none_20250612_211807/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/resnet18_none_20250612_180636/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/simple_cnn_none_20250612_184818/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/resnet18_none_20250612_203242/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/baby_cnn_none_20250612_173623/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/tiny_cnn_none_20250612_201735/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/tiny_cnn_none_20250612_175136/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/tiny_cnn_none_20250612_200233/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/simple_cnn_none_20250612_224839/all_metrics.json
Trying to open ../Tests3/SGD/checkpoints/metrics/baby_cnn_none_20250612_193214/all_metrics.json
Trying to open ../Tests3/SGD/checkpo

In [16]:
df = pd.DataFrame(all_data)
df.sort_values(by='final_val_acc', ascending=False, inplace=True)
sub_df = df.copy()
sub_df.drop(columns=['data_dir', 'checkpoint_name', 'download_weights', 'model'], inplace=True)
sub_df.drop(columns=['batch_size', 'max_epochs', 'num_workers', 'gpu_id'], inplace=True)
sub_df.drop(columns=['learning_rate', 'test_phase','noise_type'], inplace=True)
sub_df.drop(columns=['noise_layer', 'weight_decay'], inplace=True)

sub_df['noise_distribution'] = sub_df.apply(lambda row: 'baseline' if row['noise_schedule'] =='constant' else row['noise_distribution'],axis=1)

sub_df.drop(columns=['noise_schedule', 'noise_magnitude'], inplace=True)

df_selection = sub_df[['classifier', 'noise_distribution', 'optimizer', 'momentum', 'best_accuracy', 'final_val_acc', 'final_train_acc']].copy()

df_selection.sort_values(by='final_val_acc', ascending=False, inplace=True)
#df_selection = df_selection[df_selection['classifier'] == 'resnet18']
df_selection

,classifier,noise_distribution,optimizer,momentum,best_accuracy,final_val_acc,final_train_acc
31,resnet18,baseline,adam,0.0,0.9062,0.8978,0.980389
14,resnet18,gaussian,sgd,0.9,0.8542,0.8504,0.943810
32,simple_cnn,uniform,adam,0.0,0.8486,0.8486,0.959816
19,simple_cnn,baseline,sgd,0.9,0.8500,0.8480,0.940765
18,resnet18,baseline,sgd,0.9,0.8484,0.8430,0.967248
13,resnet18,uniform,sgd,0.9,0.8516,0.8426,0.944812
23,simple_cnn,gaussian,sgd,0.9,0.8426,0.8416,0.919912
30,simple_cnn,gaussian,adam,0.0,0.8486,0.8376,0.962660
27,simple_cnn,baseline,adam,0.0,0.8470,0.8370,0.963602
16,simple_cnn,uniform,sgd,0.9,0.8398,0.8352,0.919872


In [11]:
#df["best_val_acc"] = df["val_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
#df["last_val_acc"] = df["val_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
#df["best_train_acc"] = df["train_acc"].apply(lambda x : max(x) if isinstance(x, list) else x)
#df["last_train_acc"] = df["train_acc"].apply(lambda x : x[-1] if isinstance(x, list) else x)
#df["train_last_is_best"] = df["last_train_acc"] == df["best_train_acc"]
#df["val_last_is_best"] = df["last_val_acc"] == df["best_val_acc"]
#df

In [12]:
constant_cols = []
for col in df.columns:
    try:
        if df[col].nunique() == 1:
            constant_cols.append(col)
    except TypeError:
        # ignorer colonnes non hashables
        pass

print("Colonnes constantes :", constant_cols)


Colonnes constantes : []


In [15]:
df

""


In [13]:
#sub_df = df.copy()
#sub_df.drop(columns=['data_dir', 'checkpoint_name', 'download_weights', 'model'], inplace=True)
#sub_df.drop(columns=['batch_size', 'max_epochs', 'num_workers', 'gpu_id'], inplace=True)
#sub_df.drop(columns=['learning_rate', 'test_phase',], inplace=True)
#sub_df.drop(columns=['train_acc', 'val_acc', 'noise_metrics'], inplace=True)
#sub_df
#pass

KeyError: "['data_dir', 'checkpoint_name', 'download_weights', 'model'] not found in axis"

In [14]:
cols = ['last_val_acc', 'classifier', 'noise_type', 'optimizer']

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_sorted = df.sort_values(by=cols, ascending=False)
df_sorted

KeyError: 'last_val_acc'

In [22]:
df_weight = df_sorted[(df_sorted['noise_type'] == 'weight')]
df_weight = df_weight.sort_values('noise_layer', ascending=False)

In [12]:
df_exp = df_sorted[(df_sorted['noise_schedule']=='exponential') | (df_sorted['noise_schedule'] == 'constant')]
df_exp = df_exp[df_exp['classifier'] == 'resnet18']
cols = ['noise_type','noise_layer', 'momentum']
df_exp.sort_values(cols, ascending=False)

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,permanent,noise_during_stuck_only,patience,subset,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best


In [14]:
df_no_momentum = df_exp[df_exp['momentum']==0.0]
df_no_momentum

,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
4,resnet18,0.001,weight,0.01,exponential,['bias'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.855569,0.853365,0.926783,0.926002,False,False
22,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.849860,0.848758,0.901362,0.901022,False,False
3,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.775441,0.775441,0.808694,0.808694,True,True
2,resnet18,0.001,weight,0.01,exponential,['bias'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.765224,0.762119,0.862430,0.862430,True,False
25,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.764123,0.746995,0.828325,0.828325,True,False
76,resnet18,0.001,weight,0.01,exponential,['layer4.'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.764123,0.746995,0.828325,0.828325,True,False
8,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.702123,0.696715,0.748898,0.748898,True,False
75,resnet18,0.001,weight,0.01,exponential,['conv1'],gaussian,sgd,0.0,0.9,0.999,0.4,NaN,0.702123,0.696715,0.748898,0.748898,True,False
7,resnet18,0.001,none,0.01,constant,None,gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.852264,0.851462,0.926583,0.926362,False,False
13,resnet18,0.001,none,0.01,constant,None,gaussian,sgd,0.0,0.9,0.999,1.0,NaN,0.850361,0.850361,0.927364,0.927364,True,True


In [22]:
df_tinyCNN = df_sorted[df_sorted['classifier'] == 'tiny_cnn']
df_tinyCNN = df_tinyCNN[(df_tinyCNN['momentum'] == 0)|((df_tinyCNN['beta1'] == 0))]
df_tinyCNN = df_tinyCNN[df_tinyCNN['noise_type'] == 'gradient']
df_tinyCNN = df_tinyCNN[df_tinyCNN['noise_schedule'] == 'constant']
df_tinyCNN.sort_values('best_val_acc', ascending=False)




,classifier,weight_decay,noise_type,noise_magnitude,noise_schedule,noise_layer,noise_distribution,optimizer,momentum,beta1,beta2,subset,noise_layers,best_val_acc,last_val_acc,best_train_acc,last_train_acc,train_last_is_best,val_last_is_best
129,tiny_cnn,0.001,gradient,0.05,exponential,['layer4.'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.840345,0.839543,0.865705,0.865705,True,False
284,tiny_cnn,0.001,gradient,0.01,exponential,None,gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.839844,0.838842,0.870693,0.869371,False,False
239,tiny_cnn,0.001,gradient,0.01,exponential,['none'],gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.839143,0.838241,0.867969,0.867969,True,False
240,tiny_cnn,0.001,gradient,0.01,exponential,['none'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838842,0.838241,0.867428,0.866426,False,False
196,tiny_cnn,0.001,gradient,0.05,exponential,['none'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838442,0.837941,0.868770,0.868770,True,False
138,tiny_cnn,0.001,gradient,0.05,exponential,['bias'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.838241,0.837941,0.865745,0.865745,True,False
153,tiny_cnn,0.001,gradient,0.05,exponential,['conv1'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.837941,0.835938,0.864944,0.864944,True,False
219,tiny_cnn,0.001,gradient,0.05,exponential,['none'],gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.836338,0.834435,0.867889,0.867889,True,False
142,tiny_cnn,0.001,gradient,0.01,exponential,['conv1'],gaussian,adam,0.9,0.0,0.990,1.0,NaN,0.835837,0.834736,0.865365,0.863582,False,False
289,tiny_cnn,0.001,gradient,0.01,exponential,None,gaussian,adam,0.9,0.0,0.999,1.0,NaN,0.835337,0.834635,0.869371,0.869371,True,False
